In [80]:
import torch #The torch package contains data structures for multi-dimensional tensors and mathematical operations over these are defined.
import torchvision #The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score
import torch_geometric.transforms as T
import pandas as pd

In [81]:
edge_dataset=pd.read_csv('large_twitch_edges.csv')

In [89]:
features_dataset=pd.read_csv('large_twitch_features.csv')

In [83]:
edge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6797557 entries, 0 to 6797556
Data columns (total 2 columns):
 #   Column        Dtype
---  ------        -----
 0   numeric_id_1  int64
 1   numeric_id_2  int64
dtypes: int64(2)
memory usage: 103.7 MB


In [84]:
edge_dataset.head()

,numeric_id_1,numeric_id_2
0,98343,141493
1,98343,58736
2,98343,140703
3,98343,151401
4,98343,157118


In [85]:
features_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168114 entries, 0 to 168113
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   views         168114 non-null  int64 
 1   mature        168114 non-null  int64 
 2   life_time     168114 non-null  int64 
 3   created_at    168114 non-null  object
 4   updated_at    168114 non-null  object
 5   numeric_id    168114 non-null  int64 
 6   dead_account  168114 non-null  int64 
 7   language      168114 non-null  object
 8   affiliate     168114 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 11.5+ MB


In [90]:
features_dataset.head()

,views,mature,life_time,created_at,updated_at,numeric_id,dead_account,language,affiliate
0,7879,1,969,2016-02-16,2018-10-12,0,0,EN,1
1,500,0,2699,2011-05-19,2018-10-08,1,0,EN,0
2,382502,1,3149,2010-02-27,2018-10-12,2,0,EN,1
3,386,0,1344,2015-01-26,2018-10-01,3,0,EN,0
4,2486,0,1784,2013-11-22,2018-10-11,4,0,EN,0


In [91]:
features_dataset.head()

,views,mature,life_time,created_at,updated_at,numeric_id,dead_account,language,affiliate
0,7879,1,969,2016-02-16,2018-10-12,0,0,EN,1
1,500,0,2699,2011-05-19,2018-10-08,1,0,EN,0
2,382502,1,3149,2010-02-27,2018-10-12,2,0,EN,1
3,386,0,1344,2015-01-26,2018-10-01,3,0,EN,0
4,2486,0,1784,2013-11-22,2018-10-11,4,0,EN,0


In [92]:
edge_dataset.describe()

,numeric_id_1,numeric_id_2
count,6.797557e+06,6.797557e+06
mean,8.382801e+04,8.401523e+04
std,4.820513e+04,4.852719e+04
min,0.000000e+00,0.000000e+00
25%,4.221700e+04,4.204500e+04
50%,8.354600e+04,8.385100e+04
75%,1.256420e+05,1.259570e+05
max,1.681120e+05,1.681130e+05


**Converting Dataset to Pytorch Geometric Data**

In [93]:
features_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168114 entries, 0 to 168113
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   views         168114 non-null  int64 
 1   mature        168114 non-null  int64 
 2   life_time     168114 non-null  int64 
 3   created_at    168114 non-null  object
 4   updated_at    168114 non-null  object
 5   numeric_id    168114 non-null  int64 
 6   dead_account  168114 non-null  int64 
 7   language      168114 non-null  object
 8   affiliate     168114 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 11.5+ MB


In [94]:
features_dataset['language'].unique()

array(['EN', 'FR', 'KO', 'JA', 'RU', 'PL', 'DE', 'ES', 'IT', 'PT',
       'OTHER', 'TR', 'ZH', 'SV', 'NL', 'TH', 'CS', 'DA', 'HU', 'FI',
       'NO'], dtype=object)

In [95]:
languages = ['EN', 'FR', 'KO', 'JA', 'RU', 'PL', 'DE', 'ES', 'IT', 'PT',
       'OTHER', 'TR', 'ZH', 'SV', 'NL', 'TH', 'CS', 'DA', 'HU', 'FI',
       'NO']

from sklearn.preprocessing import LabelEncoder

def encode_df(dataframe):
    le = LabelEncoder()
    features_dataset['language'] = le.fit_transform(features_dataset['language'])
    return dataframe

#encode the dataframe
features_dataset = encode_df(features_dataset)
features_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168114 entries, 0 to 168113
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   views         168114 non-null  int64 
 1   mature        168114 non-null  int64 
 2   life_time     168114 non-null  int64 
 3   created_at    168114 non-null  object
 4   updated_at    168114 non-null  object
 5   numeric_id    168114 non-null  int64 
 6   dead_account  168114 non-null  int64 
 7   language      168114 non-null  int32 
 8   affiliate     168114 non-null  int64 
dtypes: int32(1), int64(6), object(2)
memory usage: 10.9+ MB


In [96]:
node_features = features_dataset[["numeric_id"]]

In [97]:
# node_features['created_at'] = pd.to_datetime(node_features['created_at']).astype('int64')/ 10**9
# node_features['updated_at'] = pd.to_datetime(node_features['updated_at']).astype('int64')/ 10**9
node_features.head()

,numeric_id
0,0
1,1
2,2
3,3
4,4


In [98]:
node_features = node_features.astype('float32')
node_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168114 entries, 0 to 168113
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   numeric_id  168114 non-null  float32
dtypes: float32(1)
memory usage: 656.8 KB


In [99]:
x =  torch.from_numpy(node_features.to_numpy())
x.shape # [num_nodes x num_features]

torch.Size([168114, 1])

In [100]:
x.dtype

torch.float32

In [101]:
features_dataset[["dead_account","language","affiliate"]] = features_dataset[["dead_account","language","affiliate"]].astype('int64')
features_dataset[["dead_account","language","affiliate"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168114 entries, 0 to 168113
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   dead_account  168114 non-null  int64
 1   language      168114 non-null  int64
 2   affiliate     168114 non-null  int64
dtypes: int64(3)
memory usage: 3.8 MB


In [102]:
# Select node features
labels = features_dataset[["language"]]
labels.head()

# labels = labels.astype(float)

# Convert to numpy
y =  torch.from_numpy(labels.to_numpy())
y.shape # [num_nodes, 1] --> node regression

torch.Size([168114, 1])

In [103]:
y = y.reshape(-1,)

In [104]:
y.shape

torch.Size([168114])

In [105]:
labels.dtypes

language    int64
dtype: object

In [106]:
edge_dataset = edge_dataset.astype('int64')

In [107]:
edge_dataset.dtypes

numeric_id_1    int64
numeric_id_2    int64
dtype: object

In [108]:
edge_dataset = edge_dataset.sort_values(by=['numeric_id_1'])

In [109]:
edge_index = edge_dataset.transpose()

In [110]:
edge_index.dtypes.unique()

array([dtype('int64')], dtype=object)

In [111]:
all_edges =  torch.from_numpy(edge_index.to_numpy()) # [2, num_edges]
print(all_edges.shape)

torch.Size([2, 6797557])


In [113]:
len(features_dataset)

168114

In [114]:
train_arr = np.array([True for i in range(round(len(features_dataset)*0.85))])

In [115]:
train_arr = np.append(train_arr,np.array([False for i in range(int(len(features_dataset)*0.15))]))

In [116]:
train_arr.shape

(168114,)

In [117]:
test_arr = np.array([False for i in range(round(len(features_dataset)*0.55))])
test_arr = np.append(test_arr,np.array([True for i in range(int(len(features_dataset)*0.45))]))

In [118]:
test_arr.shape

(168114,)

In [119]:
val_arr = np.array([False for i in range(round(len(features_dataset)*0.35))])
val_arr = np.append(val_arr,np.array([True for i in range(round(len(features_dataset)*0.35))]))
val_arr = np.append(val_arr,np.array([False for i in range(int(len(features_dataset)*0.30))]))

In [120]:
val_arr.shape

(168114,)

In [121]:
train_mask = torch.from_numpy(train_arr)
test_mask = torch.from_numpy(test_arr)
val_mask = torch.from_numpy(val_arr)

In [123]:
from torch_geometric.data import Data
data = Data(x=x, edge_index=all_edges, y=y)
train_data = Data(x=x, edge_index=all_edges, y=y)
test_data = Data(x=x, edge_index=all_edges, y=y)
val_data =Data(x=x, edge_index=all_edges, y=y)

In [124]:
train_data

Data(x=[168114, 1], edge_index=[2, 6797557], y=[168114])

In [125]:
data.num_classes = 21
data.train_mask = train_mask
data.test_mask = test_mask
data.val_mask = val_mask

train_data.num_classes = 21
train_data.train_mask = train_mask
train_data.test_mask = test_mask
train_data.val_mask = val_mask

test_data.num_classes = 21
test_data.train_mask = train_mask
test_data.test_mask = test_mask
test_data.val_mask = val_mask

val_data.num_classes = 21
val_data.train_mask = train_mask
val_data.test_mask = test_mask
val_data.val_mask = val_mask

In [126]:
train_edge_label = torch.from_numpy(np.array([1.0 for i in range(round(len(features_dataset)*0.50))]))
test_edge_label = torch.from_numpy(np.append(np.array([1.0 for i in range(round(len(features_dataset)*0.2))]),np.array([0.0 for i in range(round(len(features_dataset)*0.2))])))
val_edge_label = torch.from_numpy(np.append(np.array([1.0 for i in range(round(len(features_dataset)*0.1))]), np.array([0.0 for i in range(round(len(features_dataset)*0.1))])))

In [127]:
train_edge_label_index = all_edges[:,:len(train_edge_label)]
test_edge_label_index = all_edges[:,len(train_edge_label):len(train_edge_label)+len(test_edge_label)]
val_edge_label_index = all_edges[:,len(train_edge_label)+len(test_edge_label):len(train_edge_label)+len(test_edge_label)+len(val_edge_label)]

In [128]:
len(train_edge_label_index)

2

In [129]:
train_data.edge_label = train_edge_label
train_data.edge_label_index = train_edge_label_index

test_data.edge_label = test_edge_label
test_data.edge_label_index = test_edge_label_index

val_data.edge_label = val_edge_label
val_data.edge_label_index = val_edge_label_index

In [130]:
data

Data(x=[168114, 1], edge_index=[2, 6797557], y=[168114], num_classes=21, train_mask=[168114], test_mask=[168114], val_mask=[168114])

In [131]:
train_data

Data(x=[168114, 1], edge_index=[2, 6797557], y=[168114], num_classes=21, train_mask=[168114], test_mask=[168114], val_mask=[168114], edge_label=[84057], edge_label_index=[2, 84057])

In [132]:
test_data

Data(x=[168114, 1], edge_index=[2, 6797557], y=[168114], num_classes=21, train_mask=[168114], test_mask=[168114], val_mask=[168114], edge_label=[67246], edge_label_index=[2, 67246])

In [133]:
val_data

Data(x=[168114, 1], edge_index=[2, 6797557], y=[168114], num_classes=21, train_mask=[168114], test_mask=[168114], val_mask=[168114], edge_label=[33622], edge_label_index=[2, 33622])

In [135]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

In [143]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        #return the indices of a non-zero element
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [146]:
model = Net(data.num_features, 128, 64).to('cpu')
train_data = train_data.to('cpu')
test_data = test_data.to('cpu')
val_data = val_data.to('cpu')

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [147]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 5            |        cudaMalloc retries: 6         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    7401 MB |    7401 MB |    7795 MB |  404241 KB |
|       from large pool |    7396 MB |    7396 MB |    7785 MB |  397455 KB |
|       from small pool |       4 MB |       4 MB |      10 MB |    6785 KB |
|---------------------------------------------------------------------------|
| Active memory         |    7401 MB |    7401 MB |    7795 MB |  404241 KB |
|       from large pool |    7396 MB |    7396 MB |    7785 MB |

In [148]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1))

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss.item()


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())



In [149]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

Epoch: 001, Loss: 2478858054.3100, Val: 0.5000, Test: 0.5000
Epoch: 002, Loss: 722795190.2349, Val: 0.5000, Test: 0.5000
Epoch: 003, Loss: 271352633.0505, Val: 0.5000, Test: 0.5000
Epoch: 004, Loss: 368072025.4063, Val: 0.5000, Test: 0.5000
Epoch: 005, Loss: 489682437.4896, Val: 0.5000, Test: 0.5000
Epoch: 006, Loss: 467447667.7303, Val: 0.5000, Test: 0.5000
Epoch: 007, Loss: 364682182.4178, Val: 0.5000, Test: 0.5000
Epoch: 008, Loss: 258917138.2727, Val: 0.5000, Test: 0.5000
Epoch: 009, Loss: 186490638.6461, Val: 0.5000, Test: 0.5000
Epoch: 010, Loss: 141341722.5309, Val: 0.5000, Test: 0.5000
Epoch: 011, Loss: 112819329.2889, Val: 0.5000, Test: 0.5000
Epoch: 012, Loss: 91142598.6322, Val: 0.5000, Test: 0.5000
Epoch: 013, Loss: 75603745.2195, Val: 0.5000, Test: 0.5000
Epoch: 014, Loss: 64989690.7189, Val: 0.5000, Test: 0.5000
Epoch: 015, Loss: 59004497.5192, Val: 0.5000, Test: 0.5000
Epoch: 016, Loss: 56401925.6622, Val: 0.5000, Test: 0.5000
Epoch: 017, Loss: 55014082.6201, Val: 0.5000

KeyboardInterrupt: 

In [61]:
torch.cuda.empty_cache()